In [1]:
import os
import shutil

print(" Cleaning up old files to prevent errors...")
shutil.rmtree("/kaggle/working/dataset", ignore_errors=True)


!pip install -q gdown


files_to_download = {
    "Train_Part1.zip": "1QtSOBpyXyqiGRNEFsYZUe2Ln2j3zwp2E",
    "Train_Part2.zip": "1_HDaXfDzxpgF8ESDpG1dU4aOoLt8x08k",
    "Train_Part3.zip": "1Pcd6nNwAaVttyCDGK60BnoAFhhwtypq_",
    "Validation.zip":  "1lI3Y3UsDlsrYUMQWvd9O7L6QwgcY4pjx"
}
# ==========================================


output_dir = "/kaggle/working/dataset"
os.makedirs(output_dir, exist_ok=True)

print(f" Starting fresh download of {len(files_to_download)} files...")

for filename, file_id in files_to_download.items():
    print(f"   Downloading {filename}...")
    !gdown --id {file_id} -O {filename}
    
    print(f"   Unzipping {filename}...")
    !unzip -qo {filename} -d {output_dir}
    
    os.remove(filename)

print(" All files downloaded and merged successfully!")
print(f"📂 Data is located at: {output_dir}") 


 Cleaning up old files to prevent errors...
 Starting fresh download of 4 files...
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1QtSOBpyXyqiGRNEFsYZUe2Ln2j3zwp2E
From (redirected): https://drive.google.com/uc?id=1QtSOBpyXyqiGRNEFsYZUe2Ln2j3zwp2E&confirm=t&uuid=28e2de00-31c0-4cd4-8740-da5fccfb9675
To: /kaggle/working/Train_Part1.zip
100%|██████████████████████████████████████| 2.15G/2.15G [00:23<00:00, 90.9MB/s]
   Unzipping Train_Part1.zip...
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1_HDaXfDzxpgF8ESDpG1dU4aOoLt8x

In [2]:
!pip install -q "monai[nibabel, tqdm]" torch torchvision
print(" Libraries installed!")

import os
import glob
import torch
import numpy as np
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, Orientationd, Spacingd,
    NormalizeIntensityd, RandCropByPosNegLabeld, RandFlipd, MapTransform
)
from monai.networks.nets import SegResNet
from monai.losses import DiceLoss
from monai.data import CacheDataset, DataLoader
from monai.inferers import sliding_window_inference


data_dir = '/kaggle/working/dataset/Healing_Algorithms_Training_Data'
files = glob.glob(os.path.join(data_dir, "HelioGLI*"))
print(f" Training Data Directory: {data_dir}")
print(f" Found {len(files)} patients. Ready to train!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 28.2 MB/s eta 0:00:00a 0:00:01
 Libraries installed!


/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():
2025-12-30 11:24:40.173356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767093880.342352      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767093880.389361      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767093880.818504      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767093880.818541      55 computation_placer.cc:177] computation placer already registered.

 Training Data Directory: /kaggle/working/dataset/Healing_Algorithms_Training_Data
 Found 236 patients. Ready to train!


In [3]:
import os
import glob
import torch
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, Orientationd, Spacingd,
    NormalizeIntensityd, RandCropByPosNegLabeld, RandFlipd, MapTransform
)
from monai.data import Dataset, DataLoader

data_dir = '/kaggle/working/dataset'

class ConvertToBratsClassesd(MapTransform):
    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            result = []
            
            result.append(torch.logical_or(d[key] == 2, d[key] == 4))
            result.append(torch.logical_or(torch.logical_or(d[key] == 2, d[key] == 4), d[key] == 1))
            result.append(d[key] == 2)
            d[key] = torch.stack(result, axis=0).float()
        return d

train_transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image"]),
    ConvertToBratsClassesd(keys=["label"]),
    Orientationd(keys=["image", "label"], axcodes="RAS"),
    Spacingd(keys=["image", "label"], pixdim=(1.0, 1.0, 1.0), mode=("bilinear", "nearest")),
    NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
    RandCropByPosNegLabeld(keys=["image", "label"], label_key="label", spatial_size=(128, 128, 128), pos=1, neg=1, num_samples=1),
    RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),
])

val_transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image"]),
    ConvertToBratsClassesd(keys=["label"]),
    Orientationd(keys=["image", "label"], axcodes="RAS"),
    Spacingd(keys=["image", "label"], pixdim=(1.0, 1.0, 1.0), mode=("bilinear", "nearest")),
    NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
])

print(f" Searching for patient folders inside {data_dir}...")

patient_folders = sorted(glob.glob(os.path.join(data_dir, "**", "HelioGLI-*"), recursive=True))

if not patient_folders:
    patient_folders = sorted(glob.glob(os.path.join(data_dir, "**", "Helio*"), recursive=True))

print(f" Found {len(patient_folders)} patient folders.")


data_dicts = []

for subject_path in patient_folders:
   
    if not os.path.isdir(subject_path):
        continue
        
    all_files = os.listdir(subject_path)

    def get_file(key):
        for f in all_files:
            if key in f and f.endswith('.nii.gz'):
                return os.path.join(subject_path, f)
        return None

    try:
        t1n = get_file("t1n")
        t1c = get_file("t1c")
        t2w = get_file("t2w")
        t2f = get_file("t2f")
        seg = get_file("seg")

        if None in [t1n, t1c, t2w, t2f, seg]:
            continue

        data_dicts.append({
            "image": [t1n, t1c, t2w, t2f],
            "label": seg
        })
    except Exception:
        continue

if not data_dicts:
    raise RuntimeError(" No valid data found! Please check if your zip extracted correctly.")

split_idx = int(len(data_dicts) * 0.8)
train_files, val_files = data_dicts[:split_idx], data_dicts[split_idx:]

print(f" Data split: {len(train_files)} training, {len(val_files)} validation")

train_ds = Dataset(data=train_files, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=2)

val_ds = Dataset(data=val_files, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False)

 Searching for patient folders inside /kaggle/working/dataset...
 Found 604 patient folders.
 Data split: 483 training, 121 validation


/usr/local/lib/python3.12/dist-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.spatial.dictionary Orientationd.__init__:labels: Current default value of argument `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` was changed in version None from `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` to `labels=None`. Default value changed to None meaning that the transform now uses the 'space' of a meta-tensor, if applicable, to determine appropriate axis labels.
  warn_deprecated(argname, msg, warning_category)


In [4]:
import torch
import numpy as np
from monai.networks.nets import SegResNet
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SegResNet(
    spatial_dims=3, in_channels=4, out_channels=3, 
    init_filters=16, dropout_prob=0.2
).to(device)

loss_function = DiceLoss(smooth_nr=0, smooth_dr=1e-5, squared_pred=True, to_onehot_y=False, sigmoid=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-4, weight_decay=1e-5)
scaler = torch.amp.GradScaler('cuda')

max_epochs = 5
val_interval = 1
best_metric = -1
best_metric_epoch = -1

print(f" Phase 1: Warming up for {max_epochs} epochs...")

for epoch in range(max_epochs):
    model.train()
    epoch_loss = 0
    step = 0
    
    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data["image"].to(device), batch_data["label"].to(device)
        optimizer.zero_grad()
        
        with torch.amp.autocast('cuda'):
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{max_epochs}, Train Loss: {epoch_loss/step:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            dice_vals = []
            for val_data in val_loader:
                val_inputs, val_labels = val_data["image"].to(device), val_data["label"].to(device)
                val_outputs = sliding_window_inference(val_inputs, (128, 128, 128), 4, model, overlap=0.5)
                val_outputs = (val_outputs.sigmoid() > 0.5).float()
                dice = (2 * (val_outputs * val_labels).sum()) / (val_outputs.sum() + val_labels.sum() + 1e-5)
                dice_vals.append(dice.item())
            
            mean_dice = np.mean(dice_vals)
            print(f"Validation Dice: {mean_dice:.4f}")
            
        
            if mean_dice > best_metric:
                best_metric = mean_dice
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), "best_metric_model.pth")
                print(" Model Saved!")

 Phase 1: Warming up for 5 epochs...
Epoch 1/5, Train Loss: 0.8261


/usr/local/lib/python3.12/dist-packages/monai/inferers/utils.py:226: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:306.)
  win_data = torch.cat([inputs[win_slice] for win_slice in unravel_slice]).to(sw_device)
/usr/local/lib/python3.12/dist-packages/monai/inferers/utils.py:370: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:306.)
  out

Validation Dice: 0.1583
 Model Saved!
Epoch 2/5, Train Loss: 0.7371
Validation Dice: 0.5002
 Model Saved!
Epoch 3/5, Train Loss: 0.6357
Validation Dice: 0.7096
 Model Saved!
Epoch 4/5, Train Loss: 0.5107
Validation Dice: 0.7413
 Model Saved!
Epoch 5/5, Train Loss: 0.4016
Validation Dice: 0.7464
 Model Saved!


In [5]:
import os
import torch
import numpy as np
from monai.inferers import sliding_window_inference
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, Orientationd, Spacingd,
    NormalizeIntensityd, RandCropByPosNegLabeld, RandFlipd
)
from monai.data import Dataset, DataLoader

max_epochs = 10
val_interval = 1
spatial_size = (128, 128, 128)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image"]),
    ConvertToBratsClassesd(keys=["label"]),
    Orientationd(keys=["image", "label"], axcodes="RAS"),
    Spacingd(keys=["image", "label"], pixdim=(1.0, 1.0, 1.0), mode=("bilinear", "nearest")),
    NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
    RandCropByPosNegLabeld(keys=["image", "label"], label_key="label", spatial_size=spatial_size, pos=1, neg=1, num_samples=1),
    RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),
])

train_ds = Dataset(data=train_files, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=2)

if os.path.exists("best_metric_model.pth"):
    model.load_state_dict(torch.load("best_metric_model.pth"))

optimizer = torch.optim.Adam(model.parameters(), 1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epochs)
scaler = torch.amp.GradScaler('cuda')

best_metric = 0.7786

for epoch in range(max_epochs):
    model.train()
    epoch_loss = 0
    step = 0

    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data["image"].to(device), batch_data["label"].to(device)
        optimizer.zero_grad()

        with torch.amp.autocast('cuda'):
            outputs = model(inputs)
            loss = loss_function(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        epoch_loss += loss.item()

    scheduler.step()
    print(f"Epoch {epoch + 1}/{max_epochs}, Train Loss: {epoch_loss/step:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            dice_vals = []
            for val_data in val_loader:
                val_inputs, val_labels = val_data["image"].to(device), val_data["label"].to(device)
                
                val_outputs = sliding_window_inference(val_inputs, (128, 128, 128), 4, model, overlap=0.5)
                val_outputs = (val_outputs.sigmoid() > 0.5).float()
                
                dice = (2 * (val_outputs * val_labels).sum()) / (val_outputs.sum() + val_labels.sum() + 1e-5)
                dice_vals.append(dice.item())

            mean_dice = np.mean(dice_vals)
            print(f"Validation Dice: {mean_dice:.4f}")

            if mean_dice > best_metric:
                best_metric = mean_dice
                torch.save(model.state_dict(), "best_metric_model.pth")
                print("New Best Model Saved!")

Epoch 1/10, Train Loss: 0.3423
Validation Dice: 0.6943
Epoch 2/10, Train Loss: 0.3036
Validation Dice: 0.7904
New Best Model Saved!
Epoch 3/10, Train Loss: 0.2789
Validation Dice: 0.7895
Epoch 4/10, Train Loss: 0.2667
Validation Dice: 0.8014
New Best Model Saved!
Epoch 5/10, Train Loss: 0.2482
Validation Dice: 0.8138
New Best Model Saved!
Epoch 6/10, Train Loss: 0.2351
Validation Dice: 0.7870
Epoch 7/10, Train Loss: 0.2225
Validation Dice: 0.8244
New Best Model Saved!
Epoch 8/10, Train Loss: 0.2284
Validation Dice: 0.8341
New Best Model Saved!
Epoch 9/10, Train Loss: 0.2445
Validation Dice: 0.8387
New Best Model Saved!
Epoch 10/10, Train Loss: 0.2257
Validation Dice: 0.8410
New Best Model Saved!
